In [1]:
import os
from pathlib import Path

import pandas as pd

from util import with_spotify_track_metadata

diff_dataset = True
data_path = Path("../../data").resolve()
os.makedirs(data_path, exist_ok=True)

In [ ]:
def fetch_song_chart_by_month():
    """Chart 2000 - 2021: the top 50 chart for every month from Jan 2000 to Jun 2021
    Note: song and albums ranked weekly by indicative revenue globally over a certain period. Indicative Revenue is a model for global sales volume, adjusted for inflation and currency exchange, and using IFPI estimated annual revenue for the recorded music industry in each country.
    #songs_chart_by_month = pd.read_csv("https://chart2000.com/data/chart2000-songmonth-0-3-0063.csv")
    """
    return pd.read_csv("/Users/pez/Downloads/chart2000-albummonth-0-3-0063.csv")


songs_chart_by_month = fetch_song_chart_by_month()
songs_chart_by_month = with_spotify_track_metadata(songs_chart_by_month)
songs_chart_by_month.to_parquet(data_path / "raw/song_chart_by_month.pq", index=False)
songs_chart_by_month.to_csv(data_path / "grammy_recordings.csv.gz", compression="gzip", index=False)